In [1]:
import zfit
import numpy as np
import matplotlib.pyplot as plt
from iminuit import Minuit
import warnings
import os
from itertools import combinations
import pandas as pd
from PDFs import (FullAngular_Physical_PDF, FullAngular_Transformed_PDF, get_inverse_values, apply_transformation_equations, get_physical_region_scan)
import matplotlib.pyplot as plt
import cmsstyle
import sys
from plot_tools import create_axes_for_pulls, plot_model 
import plot_tools 
import customPDFs
import mass_models
import tensorflow as tf
from PIL import Image
import seaborn as sns
import os, sys
import common_tools
from scipy.stats import gaussian_kde
from matplotlib.gridspec import GridSpec
from scipy import stats
import matplotlib.pyplot as plt
import mplhep as hep
from scipy.optimize import minimize, Bounds
from itertools import combinations
import math
import time
from scipy.stats import chi2 as _chi2
from scipy.integrate import quad
from zfit import z
hep.style.use("CMS")
from scipy.stats import chi2

/home/ghcp/miniconda3/envs/haza_wokr_env/lib/python3.8/site-packages/zfit/__init__.py:63: UserWarning: TensorFlow warnings are by default suppressed by zfit. In order to show them, set the environment variable ZFIT_DISABLE_TF_WARNINGS=0. In order to suppress the TensorFlow warnings AND this warning, set ZFIT_DISABLE_TF_WARNINGS=1.
  warnings.warn(


Welcome to JupyROOT 6.30/04


/home/ghcp/miniconda3/envs/haza_wokr_env/lib/python3.8/site-packages/zfit/core/basemodel.py:199: UserWarning: For the future, also decorate _pdf with @supports and specify what you support (such as 'norm=True' to keep the same behavior as before)
  warnings.warn(


# Funciones importante

In [2]:
zfit.settings.set_seed(42)
np.random.seed(42)
warnings.simplefilter('ignore')


def calculate_jacobian_numerical(func, params, keys_out, epsilon=1e-5):
    """Calcula la matriz Jacobiana numéricamente."""
    n_params = len(params)
    n_out = len(keys_out)
    J = np.zeros((n_out, n_params))
    
    def func_wrapper(p_args):
        res_dict = func(*p_args)
        return np.array([res_dict[k] for k in keys_out])

    for i in range(n_params):
        p_plus = np.copy(params)
        p_minus = np.copy(params)
        p_plus[i] += epsilon
        p_minus[i] -= epsilon
        
        f_plus = func_wrapper(p_plus)
        f_minus = func_wrapper(p_minus)
        
        deriv = (f_plus - f_minus) / (2 * epsilon)
        J[:, i] = deriv
    return J


# Generación de MC

In [3]:

#MAIN function 
folder_trans = "Plots/Transformed_Space_vFinal"
folder_phys_zoom = "Plots/Physical_Space_Zoom_vFinal"
folder_phys_full = "Plots/Physical_Space_Full_vFinal"

for f in [folder_trans, folder_phys_zoom, folder_phys_full]:
    os.makedirs(f, exist_ok=True)

print(f">>> Directorios listos:\n    - {folder_trans}\n    - {folder_phys_zoom}\n    - {folder_phys_full}")
# GENERACIÓN (toys) ---
obs = zfit.Space('cosThetaL', limits=(-1, 1)) * zfit.Space('cosThetaK', limits=(-1, 1)) * zfit.Space('phi', limits=(-np.pi, np.pi))

# Valores verdaderos físicos lhcb
true_vals_phys = [0.684, 0.014, 0.029, 0.050, -0.145, -0.136, -0.204, 0.077]
phys_keys = ['FL', 'S3', 'S9', 'AFB', 'S4', 'S7', 'S5', 'S8']
true_dict = dict(zip(phys_keys, true_vals_phys))

print(">>> Generando datos (Toy MC)...")
pdf_gen = FullAngular_Physical_PDF(obs, *true_vals_phys)
sampler = pdf_gen.create_sampler(n=2000) 
sampler.resample()


>>> Directorios listos:
    - Plots/Transformed_Space_vFinal
    - Plots/Physical_Space_Zoom_vFinal
    - Plots/Physical_Space_Full_vFinal
>>> Generando datos (Toy MC)...


Fit in transformed space

In [ ]:

raw_init = get_inverse_values(true_vals_phys)
r_keys = ['rFL', 'rS3', 'rS9', 'rAFB', 'rS4', 'rS7', 'rS5', 'rS8']
param_names_fit = [f"{k}_fit" for k in r_keys]

true_vals_trans_dict = dict(zip(param_names_fit, raw_init))

params = {k: zfit.Parameter(p_name, v, step_size=0.01) for k, p_name, v in zip(r_keys, param_names_fit, raw_init)}
pdf_fit = FullAngular_Transformed_PDF(obs, params['rFL'], params['rS3'], params['rS9'], params['rAFB'],params['rS4'], params['rS7'], params['rS5'], params['rS8'])

print(">>> 2. Ejecutando Minimización (Minuit)...")
nll = zfit.loss.UnbinnedNLL(model=pdf_fit, data=sampler)
minimizer = zfit.minimize.Minuit(tol=0.01) 
result = minimizer.minimize(nll)
m = result.info['minuit'] 

print(">>> 3. Calculando Errores MINOS...")
m.minos() 

# =======================================
# ESPACIO TRANSFORMADO (VALORES DE MINUIT)
# =======================================
print("\n" + "="*80)
print(">>> Errores MINOS (Espacio Transformado)")
print("="*80)
print(f"{'PARAM':<10} | {'VALOR':<10} | {'ERROR -':<10} | {'ERROR +':<10} | {'VERDAD':<10}")
print("-" * 80)

# guarda los valores CORRECTOS de Minuit para usarlos después
minuit_best_values = []

for rk, pname in zip(r_keys, param_names_fit):
    val = m.values[pname]
    minuit_best_values.append(val)
    
    err_low = m.merrors[pname].lower  
    err_high = m.merrors[pname].upper 
    truth = true_vals_trans_dict[pname]
    
    print(f"{rk:<10} | {val:.4f}     | {err_low:.4f}     | +{err_high:.4f}    | {truth:.4f}")
print("="*80 + "\n")

# ===========================================
# ESPACIO FÍSICO (CORREGIDO PARA USAR MINUIT)
# ===========================================
print("\n" + "="*80)
print(">>> Errores Gaussianos (Matriz de Covarianza Propagada)")
print("="*80)

# recupera la matriz de covarianza
param_objs_ordered = [params[k] for k in r_keys]
cov_trans = result.covariance(params=param_objs_ordered)

# IMPORTANTE usar los valores de MINUIT, no los de zfit.Parameter (revisar pq)
best_fit_r_values = np.array(minuit_best_values)

# Jacobiano y Propagación
J = calculate_jacobian_numerical(apply_transformation_equations, best_fit_r_values, phys_keys)
cov_phys = J @ cov_trans @ J.T
phys_errors_sigma = np.sqrt(np.diag(cov_phys))
phys_errors_dict = dict(zip(phys_keys, phys_errors_sigma))

# 4. Tabla (Calculada con los valores de Minuit)
print(f"{'PARAM':<10} | {'VALOR':<10} | {'ERROR (+/-)':<15} | {'VERDAD':<10}")
print("-" * 80)

best_fit_phys_dict = apply_transformation_equations(*best_fit_r_values)

for k in phys_keys:
    val = best_fit_phys_dict[k]
    err = phys_errors_dict[k]
    tru = true_dict[k]
    print(f"{k:<10} | {val:.4f}     | +/- {err:.4f}      | {tru:.4f}")
print("="*80 + "\n")


>>> 2. Ejecutando Minimización (Minuit)...
>>> 3. Calculando Errores MINOS...

>>> Errores MINOS (Espacio Transformado)
PARAM      | VALOR      | ERROR -    | ERROR +    | VERDAD    
--------------------------------------------------------------------------------
rFL        | 0.3667     | -0.0334     | +0.0346    | 0.3861
rS3        | 0.0596     | -0.1200     | +0.1216    | 0.0888
rS9        | 0.1201     | -0.1300     | +0.1391    | 0.1864
rAFB       | 0.0773     | -0.0590     | +0.0571    | 0.2189
rS4        | -0.5490     | -0.1600     | +0.1373    | -0.7812
rS7        | -0.3659     | -0.0888     | +0.0734    | -0.4293
rS5        | -0.3709     | -0.0594     | +0.0553    | -0.4484
rS8        | 0.3803     | -0.1192     | +0.1356    | 0.3654


>>> Errores Gaussianos (Matriz de Covarianza Propagada)
PARAM      | VALOR      | ERROR (+/-)     | VERDAD    
--------------------------------------------------------------------------------
FL         | 0.6755     | +/- 0.0149      | 0.6840
S3   

# Allowed region 

In [ ]:
def constraints(params):
    """
    Mapeo de las 5 Desigualdades Físicas.
    """
    FL, S3, S9, AFB, S4, S7, S5, S8 = params
    
    # --- Desigualdad 1: 0 <= FL <= 1 ---
    # Nota: Aunque esta se suele manejar en 'bounds', si queremos ser
    # rigurosos en esta función, podemos retornarla, aunque el optimizador
    # ya la fuerce en los límites de caja.
    # Se desdobla en dos: FL >= 0 y 1 - FL >= 0
    # Pero para el conteo de desigualdades complejas, nos enfocamos en las acopladas:
    
    # |S3| <= 0.5 * (1 - FL)
    # equivale a: (0.5*(1-FL))^2 - S3^2 >= 0
    c2 = 0.25 * (1 - FL)**2 - S3**2
    
    # S3^2 + 4/9 AFB^2 + S9^2 <= 1/4 (1 - FL)^2
    # Pasamos todo a un lado: RHS - LHS >= 0
    c3 = 0.25 * (1 - FL)**2 - (S3**2 + (4.0/9.0)*AFB**2 + S9**2)
    
    # 4 S4^2 + S7^2 <= FL (1 - FL - 2 S3)
    # RHS - LHS >= 0
    c4 = FL * (1 - FL - 2*S3) - (4*S4**2 + S7**2)
    
    # S5^2 + 4 S8^2 <= FL (1 - FL + 2 S3)
    # RHS - LHS >= 0
    c5 = FL * (1 - FL + 2*S3) - (S5**2 + 4*S8**2)
    
    return np.array([c2, c3, c4, c5])



def get_profiled_boundary(x_idx, y_idx, n_steps=60):
    
    # Encontrar los límites REALES de X permitidos 
    # En lugar de usar rangos fijos, preguntamos al optimizador hasta dónde llega X.
    
    def objective_min_x(p): return p[x_idx]
    def objective_max_x(p): return -p[x_idx]
    def phys_constraints(p): return constraints(p)
    
    cons = [{'type': 'ineq', 'fun': phys_constraints}]
    bounds_opt = [(0, 1)] + [(-1, 1)]*7
    
    # Semilla central 
    x0 = np.array([0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
    
    # Buscamos X_min real
    res_xmin = minimize(objective_min_x, x0, method='SLSQP', bounds=bounds_opt, constraints=cons, tol=1e-6)
    real_start = res_xmin.fun if res_xmin.success else 0.0
    
    # Buscamos X_max real
    res_xmax = minimize(objective_max_x, x0, method='SLSQP', bounds=bounds_opt, constraints=cons, tol=1e-6)
    real_end = -res_xmax.fun if res_xmax.success else 1.0
    
    # escanea EXACTAMENTE entre esos límites encontrados
    # Usamos un margen epsilon mucho más pequeño, solo para estabilidad
    epsilon = 1e-5 
    x_vals = np.linspace(real_start + epsilon, real_end - epsilon, n_steps)
    
    y_min_vals = []
    y_max_vals = []
    valid_x = []
    bounds = [(0, 1)] + [(-1, 1)]*7
    base_guesses = [...]

    # base_guesses = [
    #     # Centro
    #     np.array([0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
    #     # Extremos de FL
    #     np.array([0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
    #     np.array([0.99, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
    #     # Combinaciones de signos para parámetros acoplados
    #     np.array([0.3, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]),
    #     np.array([0.3, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1, -0.1]),
    #     # Punto con FL=0.5, S3 máximo/minimo permitido
    #     np.array([0.5, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
    #     np.array([0.5, -0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
    # ]
    # --- SEMILLAS ---
    base_guesses = [
        np.array([0.5,  0.0, 0,0,0,0,0,0]), # Centro
        np.array([0.5,  0.2, 0,0,0,0,0,0]), # S3 positivo
        np.array([0.5, -0.2, 0,0,0,0,0,0]), # S3 negativo
        # Semilla para FL -> 0 (Arregla los cortes en V de S9 vs S3)
        np.array([0.01, 0.0, 0,0,0,0,0,0]), 
        # Semilla para FL -> 1 (Arregla la punta del triángulo AFB vs FL)
        np.array([0.99, 0.0, 0,0,0,0,0,0]), 
    ]

    for x_val in x_vals:
        def objective_min(p): return p[y_idx]
        def objective_max(p): return -p[y_idx]
        def fix_x_constraint(p): return p[x_idx] - x_val
        def phys_constraints(p): return constraints(p)
        cons = [{'type': 'eq', 'fun': fix_x_constraint},{'type': 'ineq', 'fun': phys_constraints}]
        best_min = np.inf
        best_max = -np.inf
        found_valid_min = False
        found_valid_max = False
        
        for seed in base_guesses:
            x0 = seed.copy()
            x0[x_idx] = x_val 
            
            # buscar MÁXIMO
            res_max = minimize(objective_max, x0, method='SLSQP', bounds=bounds, constraints=cons, tol=1e-4)
            if res_max.success and np.min(constraints(res_max.x)) > -1e-5:
                val = -res_max.fun
                if val > best_max: 
                    best_max = val
                    found_valid_max = True

            # buscar MÍNIMO
            res_min = minimize(objective_min, x0, method='SLSQP', bounds=bounds, constraints=cons, tol=1e-4)
            if res_min.success and np.min(constraints(res_min.x)) > -1e-5:
                val = res_min.fun
                if val < best_min: 
                    best_min = val
                    found_valid_min = True

        if found_valid_min and found_valid_max and best_max >= best_min:
            valid_x.append(x_val)
            y_max_vals.append(best_max)
            y_min_vals.append(best_min)
    
    return np.array(valid_x), np.array(y_min_vals), np.array(y_max_vals)




def save_boundaries(n_steps=150):
    params = ['FL', 'S3', 'S9', 'AFB', 'S4', 'S7', 'S5', 'S8']
    pairs = list(combinations(enumerate(params), 2))
    os.makedirs("Plots/BoundaryData", exist_ok=True)
    boundary_results = {}
    print(f"Iniciando cálculo de {len(pairs)} fronteras...")
    
    for i, ((idx_x, name_x), (idx_y, name_y)) in enumerate(pairs):
        start = time.time()
        vx, v_min, v_max = get_profiled_boundary(idx_x, idx_y, n_steps=n_steps)
        
        # guarda en un diccionario 
        key = f"{name_x}_vs_{name_y}"
        boundary_results[key] = {'x': vx, 'min': v_min, 'max': v_max}
        elapsed = time.time() - start
        print(f"[{i+1}/28] {key} calculado en {elapsed:.2f}s")

    np.savez_compressed("Plots/BoundaryData/theoretical_boundaries.npz", **boundary_results)
    print("\n¡Datos de fronteras guardados en Plots/BoundaryData/theoretical_boundaries.npz!")

save_boundaries(n_steps=200)





def plot_from_saved_data(df_data):
    data = np.load("Plots/BoundaryData/theoretical_boundaries.npz", allow_pickle=True)
    params = ['FL', 'S3', 'S9', 'AFB', 'S4', 'S7', 'S5', 'S8']
    latex_labels = {'FL': r'$F_L$', 'S3': r'$S_3$', 'S9': r'$S_9$', 'AFB': r'$A_{FB}$', 'S4': r'$S_4$', 'S7': r'$S_7$', 'S5': r'$S_5$', 'S8': r'$S_8$'}
    pairs = list(combinations(params, 2))
    
    output_dir = "Plots/BoundaryData"
    os.makedirs(output_dir, exist_ok=True)    
    print("Generando ploteos con estilo CMS...")

    for name_x, name_y in pairs:
        key = f"{name_x}_vs_{name_y}"
        if key not in data: continue
        res = data[key].item()
        vx, v_min, v_max = res['x'], res['min'], res['max']
        

        fig, ax = plt.subplots(figsize=(10, 10))
        ax.scatter(df_data[name_x], df_data[name_y], s=1, alpha=1, c='b', label='Monte Carlo', rasterized=True, marker='o', linewidths=0)   
        ax.plot(vx, v_max, color='red', linestyle='--', lw=2, label='Theoretical Limit')
        ax.plot(vx, v_min, color='red', linestyle='--', lw=2)
        #ax.fill_between(vx, v_min, v_max, color='red', alpha=0.2, label='Physical Region')
        #ax.fill_between( vx, v_min, v_max, facecolor="none", edgecolor="red", hatch='////', alpha=0.5, label='Physical Region')
        ax.set_xlabel(latex_labels[name_x], fontsize=12)
        ax.set_ylabel(latex_labels[name_y], fontsize=12)
        ax.grid(True, linestyle=':', alpha=0.4)
        if name_x == 'FL': ax.set_xlim(0, 1)
        if name_y == 'FL': ax.set_ylim(0, 1)
        ax.legend(loc='best', fontsize=12, frameon=True, framealpha=0.8, edgecolor='white')
        ax.tick_params(axis='both', which='major', labelsize=20, length=10, width=1)
        ax.tick_params(axis='both', which='minor', length=5, width=1)
        hep.cms.label(data=False, rlabel="Theoretical Phase Space", loc=0, ax=ax)
        plt.tight_layout()
        plt.savefig(f"{output_dir}/{key}_CMS.png", dpi=150, bbox_inches='tight')
        plt.close(fig)

    print(f"Hecho. Gráficas CMS guardadas en: {output_dir}")



#GENERACIÓN DEL MAPA FÍSICO
df_phys_region = get_physical_region_scan(n_points=1000000)
indices = list(range(8))
pairs_indices = list(combinations(indices, 2))
total_plots = len(pairs_indices)

if 'df_phys_region' in locals():
    plot_from_saved_data(df_phys_region)

Iniciando cálculo de 28 fronteras...
[1/28] FL_vs_S3 calculado en 1.56s
[2/28] FL_vs_S9 calculado en 5.77s
[3/28] FL_vs_AFB calculado en 5.55s
[4/28] FL_vs_S4 calculado en 4.86s
[5/28] FL_vs_S7 calculado en 3.92s
[6/28] FL_vs_S5 calculado en 3.91s
[7/28] FL_vs_S8 calculado en 4.83s
[8/28] S3_vs_S9 calculado en 4.11s
[9/28] S3_vs_AFB calculado en 3.92s
[10/28] S3_vs_S4 calculado en 5.45s
[11/28] S3_vs_S7 calculado en 5.47s
[12/28] S3_vs_S5 calculado en 5.43s
[13/28] S3_vs_S8 calculado en 5.47s
[14/28] S9_vs_AFB calculado en 4.46s
[15/28] S9_vs_S4 calculado en 6.37s
[16/28] S9_vs_S7 calculado en 6.22s
[17/28] S9_vs_S5 calculado en 6.16s
[18/28] S9_vs_S8 calculado en 6.43s
[19/28] AFB_vs_S4 calculado en 6.53s
[20/28] AFB_vs_S7 calculado en 6.42s
[21/28] AFB_vs_S5 calculado en 6.25s
[22/28] AFB_vs_S8 calculado en 11.53s
[23/28] S4_vs_S7 calculado en 8.56s
[24/28] S4_vs_S5 calculado en 9.21s
[25/28] S4_vs_S8 calculado en 9.53s
[26/28] S7_vs_S5 calculado en 8.89s
[27/28] S7_vs_S8 calculado e

# Contornos 

In [ ]:
plt.style.use(hep.style.CMS)
boundary_file = "Plots/BoundaryData/theoretical_boundaries.npz"
boundary_data = None

if os.path.exists(boundary_file):
    boundary_data = np.load(boundary_file, allow_pickle=True)
    print(f">>> Datos de fronteras cargados desde: {boundary_file}")
else:
    print(f">>> No se encontró {boundary_file}.")




for i, (idx_x, idx_y) in enumerate(pairs_indices):
    
    rx, ry = param_names_fit[idx_x], param_names_fit[idx_y]
    px, py = phys_keys[idx_x], phys_keys[idx_y]
    print(f"    [{i+1}/{total_plots}] {px} vs {py} ...", end="\r")
    contour_r = m.mncontour(rx, ry, cl=0.3935, size=50)

    # -------------------
    # Espacio Transformad
    # --------------------
    plt.figure(figsize=(10, 10))
    plt.plot(contour_r[:, 0], contour_r[:, 1], 'b-', linewidth=2, label=r'1$\sigma$ Contour')
    plt.plot(m.values[rx], m.values[ry], 's', color='blue', markersize=8, label='Best Fit', zorder=10)
    true_tx = true_vals_trans_dict[rx]
    true_ty = true_vals_trans_dict[ry]
    plt.plot(true_tx, true_ty, 'P', color='red', markersize=12, markeredgecolor='k', label='True Value', zorder=15)

    # minos Errors
    val_x = m.values[rx]; val_y = m.values[ry]
    minos_x_low = val_x + m.merrors[rx].lower; minos_x_high = val_x + m.merrors[rx].upper
    minos_y_low = val_y + m.merrors[ry].lower; minos_y_high = val_y + m.merrors[ry].upper
    plt.axvline(minos_x_low, color='gray', linestyle='--', alpha=0.5)
    plt.axvline(minos_x_high, color='gray', linestyle='--', alpha=0.5)
    plt.axhline(minos_y_low, color='gray', linestyle='--', alpha=0.5)
    plt.axhline(minos_y_high, color='gray', linestyle='--', alpha=0.5)
    width_x = minos_x_high - minos_x_low; width_y = minos_y_high - minos_y_low
    margin = 0.4
    plt.xlim(minos_x_low - width_x * margin, minos_x_high + width_x * margin)
    plt.ylim(minos_y_low - width_y * margin, minos_y_high + width_y * margin)
    plt.xlabel(rx); plt.ylabel(ry)
    hep.cms.label(data=False, loc=0, rlabel="") 
    plt.legend(loc='upper right', frameon=True)
    plt.grid(True, alpha=0.2)
    plt.savefig(f"{folder_trans}/Trans_{rx}_vs_{ry}.png", bbox_inches='tight')
    plt.close()

    # ---------------
    # espacio físico
    # ---------------
    n_pts = len(contour_r)
    r_matrix = np.tile(best_fit_r_values, (n_pts, 1))
    r_matrix[:, idx_x] = contour_r[:, 0]
    r_matrix[:, idx_y] = contour_r[:, 1]
    
    #  transformación inversa
    trans_contour_dict = apply_transformation_equations(r_matrix[:, 0], r_matrix[:, 1], r_matrix[:, 2], r_matrix[:, 3], r_matrix[:, 4], r_matrix[:, 5], r_matrix[:, 6], r_matrix[:, 7])
    cx_phys = trans_contour_dict[px]
    cy_phys = trans_contour_dict[py]

    # errores geometricos
    x_min_cont = np.min(cx_phys); x_max_cont = np.max(cx_phys)
    y_min_cont = np.min(cy_phys); y_max_cont = np.max(cy_phys)
    bf_x = best_fit_phys_dict[px]
    bf_y = best_fit_phys_dict[py]
    err_x_up = x_max_cont - bf_x; err_x_down = bf_x - x_min_cont
    err_y_up = y_max_cont - bf_y; err_y_down = bf_y - y_min_cont

    # datos de frontera para cada par
    boundary_x, boundary_min, boundary_max = None, None, None
    if boundary_data is not None:
        key_pair = f"{px}_vs_{py}"
        if key_pair in boundary_data:
            res = boundary_data[key_pair].item()
            boundary_x = res['x']
            boundary_min = res['min']
            boundary_max = res['max']

    def plot_physical(view_mode, x_lims=None, y_lims=None):
        fig, ax = plt.subplots(figsize=(10, 10))
        
        # region física permitida
        if boundary_x is not None:
            ax.plot(boundary_x, boundary_min, color='red', linestyle='--', linewidth=1.5, alpha=0.6, zorder=1)
            ax.plot(boundary_x, boundary_max, color='red', linestyle='--', linewidth=1.5, alpha=0.6, zorder=1)
            ax.fill_between(boundary_x, boundary_min, boundary_max, facecolor="none", edgecolor="red", hatch='///', alpha=0.2, zorder=1,label='Physical Region')

        # contorno y best fit
        ax.plot(cx_phys, cy_phys, color='black', linestyle='-', linewidth=2, label=r'1$\sigma$ Contour', zorder=5)
        ax.plot(bf_x, bf_y, marker='s', color='blue', markersize=8, linestyle='None', label='Best Fit', zorder=10)
        ax.plot(true_dict[px], true_dict[py], marker='P', color='red', markersize=12, markeredgecolor='black', linestyle='None', label='True Value', zorder=11)
        #errores contorno 
        ax.axvline(x_min_cont, color='red', linestyle=':', linewidth=1.5, alpha=0.6)
        ax.axvline(x_max_cont, color='red', linestyle=':', linewidth=1.5, alpha=0.6)
        ax.axhline(y_min_cont, color='red', linestyle=':', linewidth=1.5, alpha=0.6)
        ax.axhline(y_max_cont, color='red', linestyle=':', linewidth=1.5, alpha=0.6)
        ax.set_xlabel(px, fontsize=24)
        ax.set_ylabel(py, fontsize=24)
        hep.cms.label(data=False, loc=0, ax=ax, rlabel="")
        
        # Texto 
        res_text = (rf"${px} = {bf_x:.4f}^{{+{err_x_up:.4f}}}_{{-{err_x_down:.4f}}}$" + "\n" +rf"${py} = {bf_y:.4f}^{{+{err_y_up:.4f}}}_{{-{err_y_down:.4f}}}$")
        if view_mode == 'Zoom':
            ax.text(0.05, 0.93, res_text, transform=ax.transAxes, fontsize=18, verticalalignment='top', horizontalalignment='left',bbox=dict(facecolor='white', alpha=0.8, edgecolor='none', pad=0.5))
            if x_lims: ax.set_xlim(x_lims)
            if y_lims: ax.set_ylim(y_lims)
            
        else: # Full View
            ax.text(0.05, 0.93, res_text, transform=ax.transAxes, fontsize=16, verticalalignment='top', horizontalalignment='left', bbox=dict(facecolor='white', alpha=0.8, edgecolor='none'))
            ax.text(0.95, 0.93, "Full Range", transform=ax.transAxes, fontsize=16, verticalalignment='top', horizontalalignment='right', color='gray')
            # límites Full
            if boundary_x is not None:
                ax.set_xlim(0 if px == 'FL' else -1.05, 1 if px == 'FL' else 1.05)
                ax.set_ylim(0 if py == 'FL' else -1.05, 1 if py == 'FL' else 1.05)
            else:
                ax.set_xlim(-1.1, 1.1); ax.set_ylim(-1.1, 1.1)
                if px == 'FL': ax.set_xlim(-0.1, 1.1)
                if py == 'FL': ax.set_ylim(-0.1, 1.1)
        ax.legend(loc='best', fontsize=16, frameon=True, framealpha=0.8, edgecolor='white')        
        ax.grid(True, linestyle=':', alpha=0.4)  
        folder = folder_phys_zoom if view_mode == 'Zoom' else folder_phys_full
        # bbox_inches='tight'           para que no se corten las etiquetas grandes de CMS
        plt.savefig(f"{folder}/Phys_{px}_vs_{py}_{view_mode}.png", bbox_inches='tight', dpi=150)
        plt.close()

    # Definir límites de zoom
    mx = (x_max_cont - x_min_cont) * 0.4
    my = (y_max_cont - y_min_cont) * 0.4
    plot_physical('Zoom', x_lims=(x_min_cont-mx, x_max_cont+mx), y_lims=(y_min_cont-my, y_max_cont+my))
    plot_physical('Full')
print("\n>>> Generación de plots completada.")



>>> Datos de fronteras cargados desde: Plots/BoundaryData/theoretical_boundaries.npz
    [28/28] S5 vs S8 ....
>>> Generación de plots completada.


# Proyecciones v1

In [5]:
def pdf_cosK(x, FL):
    return 0.75 * (1.0 - FL) * (1.0 - x**2) + 1.5 * FL * x**2

def pdf_cosL(x, FL, AFB):
    return 3.0/8.0 * (1.0 + FL) + 3.0/8.0 * (1.0 - 3.0 * FL) * x**2 + AFB * x

def pdf_phi(x, S3, S9):
    mod = S3 * np.cos(2*x) + S9 * np.sin(2*x)
    return (1.0 / (2 * np.pi)) * (1.0 + mod)

# utils -------------------------------------------------
def _get_data_array(sampler):
    if hasattr(sampler, "numpy"): return sampler.numpy()
    if hasattr(sampler, "values"): return sampler.values
    return np.asarray(sampler)
import numpy as np
from scipy import stats

def _calc_chi2_standardized(obs, exp, n_params):
    """
    Calcula el Chi2 de Neyman, grados de libertad (ndof), p-value y pulls.
    Lógica basada en la aproximación de Neyman:
    """
    # 1. Asegurar que trabajamos con arrays de numpy flotantes
    obs = np.asarray(obs, dtype=np.float64)
    exp = np.asarray(exp, dtype=np.float64)
    
    # Verificación de integridad básica
    if obs.shape != exp.shape:
        raise ValueError(f"Dimensión incorrecta: Obs {obs.shape} != Exp {exp.shape}")

    # 2. Máscara: Neyman requiere Observados > 0
    mask = obs > 0
    obs_m = obs[mask]
    exp_m = exp[mask]
    
    # 3. Cálculo de Chi2 (Neyman)
    # Fórmula: sum( (O - E)^2 / O )
    terms = np.square(obs_m - exp_m) / obs_m
    chi2_val = np.sum(terms)
    
    # 4. Grados de Libertad (NDOF)
    # NDOF = N_bins_con_datos - N_parametros
    n_bins_w_data = np.sum(mask)
    ndof = n_bins_w_data - n_params
    
    # 5. Cálculo del p-value con seguridad para NDOF <= 0
    if ndof > 0:
        p_val = 1 - stats.chi2.cdf(chi2_val, ndof)
    else:
        # Si tienes más parámetros que datos, el ajuste está indeterminado
        p_val = np.nan 
        print(f"Advertencia: NDOF <= 0 ({ndof}). Revisa tu binning o modelo.")

    # 6. Cálculo de Pulls
    # Inicializamos en cero para mantener la forma original del array
    pulls = np.zeros_like(obs)
    # Pull = (Obs - Exp) / error_obs
    # Solo calculamos donde mask es True para evitar warnings de división por cero
    pulls[mask] = (obs_m - exp_m) / np.sqrt(obs_m)
    
    return chi2_val, ndof, p_val, pulls
#---------------------------------------------------------
def plot_analytical_projections(sampler, params, folder_out, n_bins=30):
    os.makedirs(folder_out, exist_ok=True)
    data = _get_data_array(sampler)
    N_ev = len(data)

    # Extraemos parámetros globales para usarlos en el texto del plot
    FL  = float(params.get('FL', 0.0))
    AFB = float(params.get('AFB', 0.0))
    S3  = float(params.get('S3', 0.0))
    S9  = float(params.get('S9', 0.0))

    # Configuración: (Array, Función, LabelEje, NombreArchivo, Rango, ArgsFunc, N_Params_NDOF)
    config = [
        (data[:, 1], pdf_cosK, r'$\cos\theta_K$', 'CosThetaK', (-1, 1), 
         (FL,), 1),
        
        (data[:, 0], pdf_cosL, r'$\cos\theta_L$', 'CosThetaL', (-1, 1), 
         (FL, AFB), 2),
        
        (data[:, 2], pdf_phi,  r'$\phi$',         'Phi',       (-np.pi, np.pi), 
         (S3, S9), 2)
    ]

    results = {}

    for arr, func, xlabel, name, limits, args, params_effective in config:
        # --- A. Cálculos Matemáticos (Sin cambios) ---
        counts, edges = np.histogram(arr, bins=n_bins, range=limits)
        centers = 0.5 * (edges[:-1] + edges[1:])
        bin_widths = edges[1] - edges[0] # Necesario para tu código de plot
        
        # Modelo Esperado (Integrado)
        expected = []
        val_norm_integ, _ = quad(lambda x: func(x, *args), limits[0], limits[1]) 
        for i in range(n_bins):
            bin_integ, _ = quad(lambda x: func(x, *args), edges[i], edges[i+1])
            expected.append((bin_integ / val_norm_integ) * N_ev)
        expected = np.array(expected)
        
        # Estadística
        chi2_val, ndof, p_val, pulls = _calc_chi2_standardized(counts, expected, params_effective)
        chi2_red = chi2_val / ndof # Necesario para tu texto

        # Preparación de curva suave (Para tu código de plot)
        x_fine = np.linspace(limits[0], limits[1], 1000)
        y_fine_raw = func(x_fine, *args)
        # Normalización matemática estricta para la curva visual
        norm_fine = np.trapz(y_fine_raw, x_fine)
        # Escalado a eventos por bin: PDF * N_total * Ancho_Bin
        y_smooth_counts = (y_fine_raw / norm_fine) * N_ev * bin_widths

        # ====================================================================
        # B. SECCIÓN DE PLOTEO (Tu código insertado)
        # ====================================================================
        
        #======
        # plot
        #======
        fig = plt.figure(figsize=(9, 9))
        gs = GridSpec(2, 1, height_ratios=[3.5, 1], hspace=0.08)
        ax0 = fig.add_subplot(gs[0])
        ax1 = fig.add_subplot(gs[1])

        # datos
        y_err_data = np.sqrt(counts)
        ax0.errorbar(centers, counts, yerr=y_err_data, xerr=bin_widths/2.0, fmt='ks', markersize=4, elinewidth=1, capsize=2, label='Toy (data)')
        
        # curva continua
        ax0.plot(x_fine, y_smooth_counts, '-', linewidth=1, label='Analytical PDF', color='b')
        
        max_data = np.max(counts + y_err_data)
        max_model = np.max(y_smooth_counts) 
        y_max_plot = max(max_data, max_model)
        
        # margen 40%
        ax0.set_ylim(0, y_max_plot * 1.5) 
        ax0.set_ylabel(f'Events / bin', fontsize=14)
        ax0.set_xlim(limits)
        ax0.set_xticklabels([])
        ax0.grid(True, alpha=0.2)
        ax0.legend(loc='best', fontsize=12, frameon=False)
        hep.cms.label(data=False, loc=0, ax=ax0, rlabel="13 TeV")
        
        # anotación
        param_text = (rf'$F_L = {FL:.3f}$' + '\n' + rf'$A_{{FB}} = {AFB:.3f}$' + '\n' + rf'$S_3 = {S3:.3f}$' + '\n' + rf'$S_9 = {S9:.3f}$' + '\n' + rf'$\chi^2/\mathrm{{ndof}} = {chi2_val:.2f}/{ndof} = {chi2_red:.2f}$' + '\n' +rf'$p$-value = {p_val:.3g}')
        ax0.text(0.05, 0.95, param_text, transform=ax0.transAxes, fontsize=11, verticalalignment='top',bbox=dict( facecolor='white', alpha=0.01))
        
        #pulls
        ax1.errorbar(centers, pulls, yerr=1.0,xerr=0, fmt='ks', markersize=1, elinewidth=1.0, capsize=0)          
        ax1.axhline(0, color='black', linewidth=1.0, linestyle='-')
        ax1.axhline(0, color='black', linestyle='', linewidth=1, alpha=0.4)
        ax1.axhline(3, color='black', linestyle=':', linewidth=1, alpha=0.8, label=r'3$\sigma$')
        ax1.axhline(-3, color='black', linestyle=':', linewidth=1, alpha=0.8)
        ax1.set_xlabel(xlabel, fontsize=14)
        ax1.set_ylabel(r'Pull', fontsize=11)
        ax1.set_xlim(limits)
        ax1.set_ylim(-5.0, 5.0)
        ax1.grid(True, alpha=0.2)
        ax0.tick_params(axis='both', which='major', labelsize=14) 
        ax1.tick_params(axis='both', which='major', labelsize=14)
        
        outname = os.path.join(folder_out, f"CMS_Proj_{name}_Chi2.png")
        plt.savefig(outname, bbox_inches='tight', dpi=150)
        plt.close(fig)
        
        # ====================================================================
        
        # guardar resultados
        results[name] = {'counts': counts, 'edges': edges, 'expected': expected, 'pulls': pulls, 'chi2': float(chi2_val), 'ndof': int(ndof), 'chi2_red': float(chi2_red), 'p_value': float(p_val), 'n_events': int(N_ev), 'plot_file': outname}
        print(f"[OK] {name}: χ2/ndof = {chi2_val:.2f}/{ndof} ({chi2_red:.2f}), p = {p_val:.3g}  — plot -> {outname}")

    return results

In [6]:

results_proj = plot_analytical_projections(
    sampler=sampler,
    params=best_fit_phys_dict,
    folder_out="Plots/Analytic_Projections",n_bins=30)

print(">>> Proyecciones listas.")


[OK] CosThetaK: χ2/ndof = 22.61/29 (0.78), p = 0.794  — plot -> Plots/Analytic_Projections/CMS_Proj_CosThetaK_Chi2.png
[OK] CosThetaL: χ2/ndof = 21.48/28 (0.77), p = 0.805  — plot -> Plots/Analytic_Projections/CMS_Proj_CosThetaL_Chi2.png
[OK] Phi: χ2/ndof = 18.22/28 (0.65), p = 0.92  — plot -> Plots/Analytic_Projections/CMS_Proj_Phi_Chi2.png
>>> Proyecciones listas.


# Proyecciones V2 

In [15]:

class FullAngular_Physical_PDF(zfit.pdf.BasePDF):
    def __init__(self, obs, FL, S3, S9, AFB, S4, S7, S5, S8, name="FullAngular_Physical_PDF"):
        params = {'FL': FL, 'S3': S3, 'S9': S9, 'AFB': AFB, 'S4': S4, 'S7': S7, 'S5': S5, 'S8': S8}
        super().__init__(obs, params, name=name)
    
    def _unnormalized_pdf(self, x):
        vars_list = z.unstack_x(x)
        cos_l = vars_list[0]
        cos_k = vars_list[1]
        phi   = vars_list[2]
        
        sin_k = tf.sqrt(1.0 - cos_k**2)
        sin_l = tf.sqrt(1.0 - cos_l**2)
        sin2_k = sin_k**2
        cos2_k = cos_k**2
        sin2_l = sin_l**2
        
        cos2l_term = 2.0 * cos_l**2 - 1.0
        sin2l_term = 2.0 * sin_l * cos_l
        sin2k_term = 2.0 * sin_k * cos_k
        
        cos_phi = tf.cos(phi)
        sin_phi = tf.sin(phi)
        cos2_phi = tf.cos(2.0 * phi)
        sin2_phi = tf.sin(2.0 * phi)

        FL = self.params['FL']
        S3 = self.params['S3']
        S9 = self.params['S9']
        AFB = self.params['AFB']
        S4 = self.params['S4']
        S7 = self.params['S7']
        S5 = self.params['S5']
        S8 = self.params['S8']
        
        term1 = 0.75 * (1.0 - FL) * sin2_k
        term2 = FL * cos2_k
        term3 = 0.25 * (1.0 - FL) * sin2_k * cos2l_term
        term4 = -1.0 * FL * cos2_k * cos2l_term
        term5 = S3 * sin2_k * sin2_l * cos2_phi
        term6 = S4 * sin2k_term * sin2l_term * cos_phi
        term7 = S5 * sin2k_term * sin_l * cos_phi
        term8 = (4.0/3.0) * AFB * sin2_k * cos_l
        term9 = S7 * sin2k_term * sin_l * sin_phi
        term10 = S8 * sin2k_term * sin2l_term * sin_phi
        term11 = S9 * sin2_k * sin2_l * sin2_phi
        
        pdf = term1 + term2 + term3 + term4 + term5 + term6 + term7 + term8 + term9 + term10 + term11
        return pdf

    # Se ha eliminado _integrate explícitamente.
    # zfit ahora usará integración numérica por defecto, lo que permite
    # calcular proyecciones (integrales parciales) correctamente.

In [16]:
def plot_analytical_projections_cms(sampler, phys_params_dict, folder_out="Plots/Analytic_Projections_CMS", n_bins=30):
    """
    Genera proyecciones con estilo CMS usando zfit, pero estandarizando
    el cálculo de Chi2/ndof y Pulls con la función _calc_chi2_standardized.
    """
    if not os.path.exists(folder_out):
        os.makedirs(folder_out)

    obs_space = sampler.space
    observables = obs_space.obs  
    
    # 1. Instanciar la PDF Física con los parámetros del Best Fit
    print(f">>> Instanciando PDF física para proyecciones...")
    pdf_model = FullAngular_Physical_PDF(
        obs=obs_space,
        **phys_params_dict 
    )

    # Etiquetas LaTeX para los ejes
    latex_labels = {
        observables[0]: r'$\cos(\theta_l)$',
        observables[1]: r'$\cos(\theta_K)$',
        observables[2]: r'$\phi$'
    }

    # Mapa de parámetros libres por variable para el cálculo correcto de NDOF
    # Índice 0 (CosL): 2 params (FL, AFB)
    # Índice 1 (CosK): 1 param (FL)
    # Índice 2 (Phi) : 2 params (S3, S9)
    n_params_map = {0: 2, 1: 1, 2: 2}

    # Total de eventos (para escalar la PDF)
    n_total = sampler.n_events.numpy()

    for obs_name in observables:
        print(f"--- Procesando plot CMS para: {obs_name} ---")
        
        # A. Crear la PDF Proyectada (integrando las otras variables)
        vars_to_integrate = [var for var in observables if var != obs_name]
        limits_integration = obs_space.with_obs(vars_to_integrate)
        pdf_proj = pdf_model.create_projection_pdf(limits=limits_integration)
        
        # B. Preparar Datos (Histograma)
        idx = observables.index(obs_name)
        data_column = sampler.unstack_x()[idx]
        
        if hasattr(data_column, "numpy"):
            data_np = data_column.numpy()
        else:
            data_np = np.array(data_column)
            
        counts, bin_edges = np.histogram(data_np, bins=n_bins)
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        bin_width = bin_edges[1] - bin_edges[0]
        y_err_data = np.sqrt(counts)
        
        # C. Calcular Modelo
        lower, upper = obs_space.with_obs(obs_name).limit1d
        
        # C.1 Para el plot suave (alta resolución)
        x_fine = np.linspace(lower, upper, 400)
        y_smooth_density = pdf_proj.pdf(x_fine).numpy()
        y_smooth_counts = y_smooth_density * n_total * bin_width
        
        # C.2 Para Chi2 y Pulls (evaluado en el centro del bin - Observado vs Esperado)
        # Nota: Usamos el valor en el centro como aproximación de la integral del bin para zfit
        y_model_at_centers_density = pdf_proj.pdf(bin_centers).numpy()
        y_model_at_centers = y_model_at_centers_density * n_total * bin_width
        
        # D. ESTADÍSTICA ESTANDARIZADA
        # ==============================================================================
        # Aquí conectamos con tu función estandarizada para garantizar consistencia.
        # Recuperamos el número de parámetros libres para esta variable específica
        n_params_curr = n_params_map.get(idx, 1) 
        
        # Llamada a la función externa que ya definiste
        chi2_val, ndof, p_val, pulls = _calc_chi2_standardized(
            obs=counts, 
            exp=y_model_at_centers, 
            n_params=n_params_curr
        )
        chi2_red = chi2_val / ndof
        # ==============================================================================

        # ====== PLOTTING ESTILO CMS (Sin cambios mayores) ======
        fig = plt.figure(figsize=(9, 9))
        gs = GridSpec(2, 1, height_ratios=[3.5, 1], hspace=0.08)
        ax0 = fig.add_subplot(gs[0])
        ax1 = fig.add_subplot(gs[1])

        # --- Panel Superior (Datos + Modelo) ---
        ax0.errorbar(bin_centers, counts, yerr=y_err_data, xerr=bin_width/2.0, 
                     fmt='ks', markersize=4, elinewidth=1, capsize=2, label='Toy (data)')

        ax0.plot(x_fine, y_smooth_counts, '-', linewidth=2, label='Analytical PDF', color='b')
        
        # Límites dinámicos
        max_data = np.max(counts + y_err_data)
        max_model = np.max(y_smooth_counts)
        y_max_plot = max(max_data, max_model)
        ax0.set_ylim(0, y_max_plot * 1.5)
        
        ax0.set_xlim(lower, upper)
        ax0.set_xticklabels([]) 
        ax0.set_ylabel(f'Events / {bin_width:.2f}', fontsize=16)
        ax0.grid(True, alpha=0.2)
        ax0.legend(loc='upper right', fontsize=13, frameon=False, bbox_to_anchor=(0.98, 0.88))

        hep.cms.label(data=False, loc=0, ax=ax0, rlabel="13 TeV")

        # --- Caja de Texto con Estadísticas ---
        FL_val = phys_params_dict.get('FL', 0)
        AFB_val = phys_params_dict.get('AFB', 0)
        S3_val = phys_params_dict.get('S3', 0)
        S9_val = phys_params_dict.get('S9', 0)

        # Usamos p-value simple para evitar error de LaTeX en backend
        param_text = (
            rf'$F_L = {FL_val:.3f}$' + '\n' +
            rf'$A_{{FB}} = {AFB_val:.3f}$' + '\n' +
            rf'$S_3 = {S3_val:.3f}$' + '\n' +
            rf'$S_9 = {S9_val:.3f}$' + '\n' +
            r'----------------' + '\n' + 
            rf'$\chi^2/\mathrm{{ndof}} = {chi2_val:.1f}/{ndof} = {chi2_red:.2f}$' + '\n' +
            rf'$p$-value = {p_val:.3g}'
        )

        ax0.text(0.05, 0.92, param_text, transform=ax0.transAxes, fontsize=12, 
                 verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.9, edgecolor='#dddddd'))

        # --- Panel Inferior (Pulls) ---
        ax1.errorbar(bin_centers, pulls, 
                     yerr=1.0,           
                     xerr=0,             
                     fmt='ks',           
                     markersize=3,       
                     elinewidth=1.0,     
                     capsize=0)          
        
        ax1.axhline(0, color='black', linewidth=1.0, linestyle='-')
        ax1.axhline(3, color='gray', linestyle=':', linewidth=1, alpha=0.8) 
        ax1.axhline(-3, color='gray', linestyle=':', linewidth=1, alpha=0.8)
        
        ax1.fill_between([lower, upper], -1, 1, color='yellow', alpha=0.1) 
        ax1.fill_between([lower, upper], -2, 2, color='green', alpha=0.05) 

        xlabel = latex_labels.get(obs_name, obs_name)
        ax1.set_xlabel(xlabel, fontsize=16)
        ax1.set_ylabel(r'Pull $(\sigma)$', fontsize=13)
        ax1.set_xlim(lower, upper)
        ax1.set_ylim(-4.9, 4.9)
        ax1.grid(True, alpha=0.2)

        ax0.tick_params(axis='both', which='major', labelsize=14) 
        ax1.tick_params(axis='both', which='major', labelsize=14)

        outname = os.path.join(folder_out, f"CMS_Proj_{obs_name}.png")
        plt.savefig(outname, bbox_inches='tight', dpi=200)
        plt.close(fig)
        print(f"    -> Gráfica guardada: {outname} (Chi2/ndof={chi2_red:.2f})")

    print(">>> Todas las proyecciones estilo CMS generadas.")

# --- LLAMADA A LA FUNCIÓN ---
plot_analytical_projections_cms(
    sampler=sampler,
    phys_params_dict=best_fit_phys_dict,
    folder_out="Plots/Analytic_Projections_CMS",
    n_bins=30
)

>>> Instanciando PDF física para proyecciones...
--- Procesando plot CMS para: cosThetaL ---
Estimated integral error ( 4.7820036572242623e-06 ) larger than tolerance ( 3e-06 ), which is maybe not enough (but maybe it's also fine). You can (best solution) implement an anatytical integral (see examples in repo) or manually set a higher number on the PDF with 'update_integration_options' and increase the 'max_draws' (or adjust 'tol'). If partial integration is chosen, this can lead to large memory consumption.This is a new warning checking the integral accuracy. It may warns too often as it is Work In Progress. If you have any observation on it, please tell us about it: https://github.com/zfit/zfit/issues/new/chooseTo suppress this warning, use zfit.settings.set_verbosity(-1).
Estimated integral error ( 2.3815317593743481e-05 ) larger than tolerance ( 3e-06 ), which is maybe not enough (but maybe it's also fine). You can (best solution) implement an anatytical integral (see examples in r

# Transformed vs physics

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
import os

hep.style.use("CMS") 

def generar_plots_individuales():
    # Configuración de datos
    x_hat = np.linspace(-5, 5, 600)
    val_infimo = 0.001
    
    # CAMBIO: Se reemplazó 0.8 por 0.9 en la lista
    casos_fl = [val_infimo, 0.3, 0.5, 0.9, 1.0]
    
    # Colores para las 5 lineas
    colores = ['blue', 'cyan', 'green', 'orange', 'red']
    
    # CAMBIO: Se actualizó la etiqueta correspondiente
    labels_fl = [
        fr'$F_L={val_infimo}$', 
        r'$F_L=0.3$', 
        r'$F_L=0.5$', 
        r'$F_L=0.9$',  # Etiqueta actualizada
        r'$F_L=1.0$'
    ]
    
    # Crear carpeta de destino
    output_dir = "Plots/physical_vs_transformed"
    os.makedirs(output_dir, exist_ok=True)
    print(f"Guardando imágenes en: {output_dir}/")

    # Función auxiliar para generar y guardar un solo plot
    def generar_plot_individual(simbolo, nombre_archivo, func_limite, func_prefactor, is_fl=False):
        fig, ax = plt.subplots(figsize=(10, 10))
        hep.cms.label(loc=0, ax=ax, data=False)
        
        ax.set_xlabel(fr'$\hat{{{simbolo}}}$', fontsize=25)
        ax.set_ylabel(fr'${simbolo}$', fontsize=25)
        
        for i, fl in enumerate(casos_fl):
            color = colores[i]
            label = labels_fl[i]
            
            limite = func_limite(fl)
            amplitud = func_prefactor(fl)
            y_curve = amplitud * np.tanh(x_hat)
            
            # Caso especial para el plot de FL
            if is_fl:
                if i == 0: 
                    y_base = 0.5 * (1 + np.tanh(x_hat))
                    ax.plot(x_hat, y_base, 'k-', linewidth=2.0, label=r'Transformation')
                    ax.axhline(1, color='red', linestyle='--', alpha=0.6)
                    ax.text(4.6, 1, r'$\mathbf{1.0}$', color='black', va='bottom', fontsize=18)
                    ax.axhline(0, color='red', linestyle='--', alpha=0.6)
                    ax.text(4.6, 0, r'$\mathbf{0.0}$', color='black', va='bottom', fontsize=18)
                continue

            # Dibujar curva
            ax.plot(x_hat, y_curve, color=color, linewidth=2.0, label=label)
            
            # Dibujar límites
            if limite > 1e-4:
                ax.axhline(limite, color=color, linestyle=':', linewidth=1.5)
                ax.text(4.6, limite, r'$\mathbf{' + f'{limite:.3f}' + r'}$', color='black', va='bottom', fontsize=14)
                
                ax.axhline(-limite, color=color, linestyle=':', linewidth=1.5)
                ax.text(4.6, -limite, r'$\mathbf{-' + f'{limite:.3f}' + r'}$', color='black', va='top', fontsize=14)
            else:
                pass 

        ax.axhline(0, color='black', linewidth=0.8)
        ax.axvline(0, color='black', linewidth=0.8)
        ax.set_xlim(-4.5, 5.5)
        
        if is_fl:
             ax.set_ylim(-0.1, 1.1)
        else:
            ax.legend(loc='upper left', fontsize=13, frameon=True, fancybox=False, edgecolor='black', ncol=1)

        plt.tight_layout()
        
        # Guardar
        ruta_completa = os.path.join(output_dir, f"{nombre_archivo}.png")
        plt.savefig(ruta_completa)
        plt.close()
        print(f" -> Generado: {nombre_archivo}.png")

    # --- GENERACIÓN DE PLOTS ---

    # 1. FL
    generar_plot_individual('F_L', 'FL_Transformation',
                            lambda x: 0, lambda x: 0, is_fl=True)

    # 2. S3
    generar_plot_individual('S_3', 'S3_Transversal', 
                            func_limite=lambda fl: 0.5 * (1 - fl), 
                            func_prefactor=lambda fl: 0.5 * (1 - fl))

    # 3. AFB
    generar_plot_individual('A_{FB}', 'AFB_Transversal', 
                            func_limite=lambda fl: 0.75 * (1 - fl),
                            func_prefactor=lambda fl: 0.75 * (1 - fl))

    # 4. S9
    generar_plot_individual('S_9', 'S9_Transversal_CPodd', 
                            func_limite=lambda fl: 0.5 * (1 - fl), 
                            func_prefactor=lambda fl: 0.5 * (1 - fl))

    # 5. S5
    generar_plot_individual('S_5', 'S5_Interference_P5p', 
                            func_limite=lambda fl: np.sqrt(fl * (1 - fl)), 
                            func_prefactor=lambda fl: np.sqrt(fl * (1 - fl)))

    # 6. S7
    generar_plot_individual('S_7', 'S7_Interference_CPodd', 
                            func_limite=lambda fl: np.sqrt(fl * (1 - fl)), 
                            func_prefactor=lambda fl: np.sqrt(fl * (1 - fl)))

    # 7. S4
    generar_plot_individual('S_4', 'S4_Interference', 
                            func_limite=lambda fl: 0.5 * np.sqrt(fl * (1 - fl)), 
                            func_prefactor=lambda fl: 0.5 * np.sqrt(fl * (1 - fl)))

    # 8. S8
    generar_plot_individual('S_8', 'S8_Interference_CPodd', 
                            func_limite=lambda fl: 0.5 * np.sqrt(fl * (1 - fl)), 
                            func_prefactor=lambda fl: 0.5 * np.sqrt(fl * (1 - fl)))

if __name__ == "__main__":
    generar_plots_individuales()

Guardando imágenes en: Plots/physical_vs_transformed/
 -> Generado: FL_Transformation.png
 -> Generado: S3_Transversal.png
 -> Generado: AFB_Transversal.png
 -> Generado: S9_Transversal_CPodd.png
 -> Generado: S5_Interference_P5p.png
 -> Generado: S7_Interference_CPodd.png
 -> Generado: S4_Interference.png
 -> Generado: S8_Interference_CPodd.png


In [2]:
# import numpy as np
# import matplotlib.pyplot as plt
# import mplhep as hep
# import matplotlib.cm as cm
# import matplotlib.colors as mcolors

# # Configuración de estilo CMS
# hep.style.use("CMS")

# def graficar_scan_individual():
#     x_hat = np.linspace(-5, 5, 600)
    
#     # Rango de FL para el scan
#     casos_fl = np.array([0.0, 0.13, 0.25, 0.38, 0.47, 0.55, 0.68, 0.79, 0.92, 1.0])
    
#     # Mapa de color y normalización
#     cmap = plt.get_cmap('viridis') 
#     norm = mcolors.Normalize(vmin=0.0, vmax=1.0)

#     # --- FUNCIÓN GENERADORA DE UN SOLO PLOT ---
#     def plot_single_observable(simbolo, func_limite, func_prefactor, is_fl=False):
#         # Creamos una figura grande e independiente para cada uno
#         # figsize=(12, 10) da buen espacio para papers/slides
#         fig, ax = plt.subplots(figsize=(10, 8))
        
#         hep.cms.label(loc=0, ax=ax, data=False)
        
#         ax.set_xlabel(fr'$\hat{{{simbolo}}}$', fontsize=30)
#         ax.set_ylabel(fr'${simbolo}$', fontsize=30)
        
#         # Ejes de referencia (Grid)
#         ax.axhline(0, color='gray', linewidth=1.0, alpha=0.4, zorder=0)
#         ax.axvline(0, color='gray', linewidth=1.0, alpha=0.4, zorder=0)

#         max_val_teorico = 0.0 # Para calcular el límite del eje Y automáticamente

#         # --- BUCLE DE SCAN ---
#         for fl in casos_fl:
#             color_linea = cmap(norm(fl))
            
#             limite = func_limite(fl)
#             amplitud = func_prefactor(fl)
#             y_curve = amplitud * np.tanh(x_hat)
            
#             # Rastreamos el valor máximo para ajustar el zoom después
#             if limite > max_val_teorico:
#                 max_val_teorico = limite

#             if is_fl:
#                 pass # Se dibuja fuera del loop
#             else:
#                 ax.plot(x_hat, y_curve, color=color_linea, linewidth=2, alpha=0.9, zorder=2)
                
#                 # Asintotas solo extremos
#                 if fl == 0.0 or fl == 1.0:
#                     if limite > 1e-3:
#                         style = '--' if fl == 1.0 else ':'
#                         ax.axhline(limite, color=color_linea, linestyle=style, linewidth=1, alpha=0.8, zorder=1)
#                         ax.axhline(-limite, color=color_linea, linestyle=style, linewidth=1, alpha=0.8, zorder=1)
                        
#                         # Texto de la asintota
#                         ax.text(4.8, limite, f'{limite:.3f}', color=color_linea, 
#                                 va='bottom', ha='right', fontsize=18, fontweight='bold', zorder=5)

#         # --- LÓGICA ESPECÍFICA PARA F_L (Transformación base) ---
#         if is_fl:
#             y_base = 0.5 * (1 + np.tanh(x_hat))
#             ax.plot(x_hat, y_base, 'k-', linewidth=3, zorder=5)
#             # Límites 0 y 1
#             ax.axhline(1, color=cmap(norm(1.0)), linestyle='--', alpha=0.8)
#             ax.text(4.8, 1.02, r'$\mathbf{1.0}$', color=cmap(norm(1.0)), fontsize=18, ha='right')
#             ax.axhline(0, color=cmap(norm(0.0)), linestyle=':', alpha=0.8)
#             ax.text(4.8, -0.08, r'$\mathbf{0.0}$', color=cmap(norm(0.0)), fontsize=18, ha='right')
            
#             ax.set_ylim(-0.2, 1.2) # Fijo para FL
#         else:
#             # --- AJUSTE AUTOMÁTICO DE ESCALA Y ---
#             # Le damos un 40% extra de aire arriba y abajo del valor máximo encontrado
#             # Esto asegura que las etiquetas de texto NUNCA se corten.
#             if max_val_teorico < 0.1: max_val_teorico = 0.1 # Evitar colapso si todo es 0
#             margen = max_val_teorico * 1.4 
#             ax.set_ylim(-margen, margen)

#         ax.set_xlim(-5.0, 5.0)
        
#         # --- BARRA DE COLOR INDIVIDUAL ---
#         sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
#         sm.set_array([])
#         cbar = fig.colorbar(sm, ax=ax, pad=0.02)
#         cbar.set_label(r'$F_L$ Value', fontsize=15, rotation=270, labelpad=20)
#         cbar.ax.tick_params(labelsize=16)

#         plt.tight_layout()
#         plt.show() # Muestra este plot y espera a que lo cierres para mostrar el siguiente

#     # ==========================================
#     # GENERACIÓN DE LOS 8 GRÁFICOS
#     # ==========================================
    
#     # 1. FL
#     print("Generando F_L...")
#     plot_single_observable('F_L', lambda x: 0, lambda x: 0, is_fl=True)

#     # 2. S3
#     print("Generando S_3...")
#     plot_single_observable('S_3', 
#                    func_limite=lambda fl: 0.5 * (1 - fl), 
#                    func_prefactor=lambda fl: 0.5 * (1 - fl))

#     # 3. AFB
#     print("Generando A_FB...")
#     plot_single_observable('A_{FB}', 
#                    func_limite=lambda fl: 0.75 * (1 - fl),
#                    func_prefactor=lambda fl: 0.75 * (1 - fl))

#     # 4. S9
#     print("Generando S_9...")
#     plot_single_observable('S_9', 
#                    func_limite=lambda fl: 0.5 * (1 - fl), 
#                    func_prefactor=lambda fl: 0.5 * (1 - fl))

#     # 5. S5
#     print("Generando S_5...")
#     plot_single_observable('S_5', 
#                    func_limite=lambda fl: np.sqrt(fl * (1 - fl)), 
#                    func_prefactor=lambda fl: np.sqrt(fl * (1 - fl)))

#     # 6. S7
#     print("Generando S_7...")
#     plot_single_observable('S_7', 
#                    func_limite=lambda fl: np.sqrt(fl * (1 - fl)), 
#                    func_prefactor=lambda fl: np.sqrt(fl * (1 - fl)))

#     # 7. S4
#     print("Generando S_4...")
#     plot_single_observable('S_4', 
#                    func_limite=lambda fl: 0.5 * np.sqrt(fl * (1 - fl)), 
#                    func_prefactor=lambda fl: 0.5 * np.sqrt(fl * (1 - fl)))

#     # 8. S8
#     print("Generando S_8...")
#     plot_single_observable('S_8', 
#                    func_limite=lambda fl: 0.5 * np.sqrt(fl * (1 - fl)), 
#                    func_prefactor=lambda fl: 0.5 * np.sqrt(fl * (1 - fl)))

# if __name__ == "__main__":
#     graficar_scan_individual()